## Preparing environment


6


Mounting the shared drive

## Move to proper directory
Move to correct dir: based on where you duplicated the environment. if you connect colab with the main drive folder you don't have to change it.

Make sure in Bottle_classification dir, there is the image folder Saline_bottle_original_size

# Saline Bottle Image Dataset preparation
## Import Libraries

In [2]:
from __future__ import print_function
# Python Imaging Library (PIL) adds support for opening, manipulating, and saving different image file formats.
from PIL import Image as pil_image
# Library for Image visualization
from IPython.display import Image as disp_image
import os
# Libary for manipulation of multi-dimensional data
import numpy as np
# scikit-learn Library for machine learning and associated facilities
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


2022-09-25 18:17:44.934866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'array_to_img' from 'keras.preprocessing.image' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/preprocessing/image.py)

## Set path where origianl Dataset is stored 

In [5]:
PATH="./Saline_bottle_original_size/"

## Define a dictionary with list of labels

In [3]:
# This sections returns a dictionary with {key=label: value= class_number}
# Input to this function is list of labels needed for classification task.
# 100 means 100% bottle fill
# 80 means 80% bottle fill
# 50 means 50% bottle fill

folders=['sal_data_100','sal_data_50','sal_data_80','sal_data_empty']
list_input = ['sal_data_100','sal_data_50','sal_data_80','sal_data_empty']

def create_classes(list_input):
    classes_dict = {}
    count_dict = {}
    for i in range(len(folders)):
        key_word= folders[i]
        if key_word in list_input:
            #get the index of the list:
            index = list_input.index(key_word)
            classes_dict[key_word] = index
        else:
            count_dict[key_word] = 0
            classes_dict[key_word] = len(list_input)
    return classes_dict, count_dict
classes_dict, count_dict = create_classes(list_input)
print(classes_dict)


{'sal_data_100': 0, 'sal_data_50': 1, 'sal_data_80': 2, 'sal_data_empty': 3}


## Creating folder and subfolders (with labels) to store the resized data as per  labels

### For RGB color space

In [19]:
resize_path= os.getcwd()
print(resize_path)
resize_path=os.path.join(resize_path,"resize_sal_data_temp_del/")

/Users/belhouariayoub/Documents/ISMIN3A/Embedded IA/PROJETBELHOUARIDAHER


In [5]:
data_label_path = os.getcwd()
data_label_path=os.path.join(data_label_path,"data_label/")
print(data_label_path)

if not os.path.isdir(data_label_path):
    print("Creating the directory")
    os.makedirs(data_label_path)

if not os.path.isdir(resize_path):
    print("Creating the directory")
    os.makedirs(resize_path)
    
for label in folders:
    label_dir=os.path.join(resize_path,label)
    if not os.path.isdir(label_dir):
        os.makedirs(label_dir)

/Users/belhouariayoub/Documents/ISMIN3A/Embedded IA/PROJETBELHOUARIDAHER/data_label/


NameError: name 'resize_path' is not defined

## Print path of the resized data

In [ ]:
print(resize_path)

## Forming a file of Label/Data

### For RGB color space

In [ ]:
img_dir=os.listdir(PATH)
data_file = open("data_label/saline_data_label.txt",'w')
for dir in img_dir:
    img_path=PATH + "/" + dir
    for img in os.listdir(img_path):
        print(dir+'/'+img)
        data_file.write(dir+'/'+img+"\n")

data_file.close() 

## Reshape
You can change the color space, the format of images (RGB, YCrCb) and use the negative of images

Default: RGB with negative
if you want YCrCb or normal image uncomment the correct line indicated in the if block

### For RGB color space
Choose resolution

In [ ]:
res = 64

In [ ]:
from PIL import Image
import PIL.ImageOps  
import cv2

# open the label/data file
f = open("data_label/saline_data_label.txt", "r")
# form list from the given file
lines = f.readlines()
X = []
Y = []
# Mention the resize image shape
size=(res,res)

resize_loc_path=resize_path

for line in lines:
  try:
    # remove the \n 
    line = line.strip('\n')
    # separate label and data
    label, file_name = line.split("/")
    # folders contains all the required labels
    if label in folders:
       file_path = PATH + label + "/" + file_name
       # open the original size image
       img=pil_image.open(file_path)

       #to use negative on images 
       inverted_img = PIL.ImageOps.invert(img)

       #to convert to YCrCb
       #imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB) # or inverted_img 

       # resize the original image to the size mentioned
       img_reshape=inverted_img.resize(size) ## choose: img, imgYCC, inverted_img
       # print(img_reshape.format, img_reshape.size, img_reshape.mode)
       # Location of the resized image as per the label
       resize_loc= resize_loc_path + label + "/" + file_name
       # checks if the resized is does not exist already 
       if file_name not in resize_loc_path + label + "/":
          img_reshape.save(resize_loc)
       # converting the image object to 
       np_im = np.array(img_reshape)
       class_number=classes_dict[label] 
       X.append(np_im)
       Y.append(class_number)
       

  except Exception as e:
           print(str(e))
    
X = np.array(X)
Y = np.array(Y)
print(Y.shape)


## Forming augmented dataset

### Warning: execute agumentation can generate misalignment between colab and drive data

### For RGB color space

In [ ]:
augmented_path= os.getcwd()
print(augmented_path)
augmented_path=os.path.join(augmented_path,"resize_sal_data_augmented/")
if not os.path.isdir(augmented_path):
    print("Creating the directory")
    os.makedirs(augmented_path)
    
for label in folders:
    label_dir=os.path.join(augmented_path,label)
    if not os.path.isdir(label_dir):
        os.makedirs(label_dir)

## Print path of the augmented data

In [ ]:
print(augmented_path)

## Achieve augmentation

### WARNING: this procedure requires lots of time writing data and if you are using Drive to store images

Choose resolution


In [ ]:
res = 64

You can change the color space, the format of images (RGB, YCrCb) and use the negative of images

Default: RGB with negative
if you want YCrCb or normal image uncomment the correct line indicated in the if block

### For RGB color space

In [ ]:
# open the label/data file
f = open("data_label/saline_data_label.txt", "r")
# form list from the given file
lines = f.readlines()
X = []
Y = []
# Mention the resize image shape
size=(res,res)

augmented_loc_path=augmented_path

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for line in lines:
  try:
    # remove the \n 
    line = line.strip('\n')
    # separate label and data
    label, file_name = line.split("/")
    # folders contains all the required labels
    if label in folders:
       file_path = PATH + label + "/" + file_name
       # open the original size image
       img=pil_image.open(file_path)
       
       #to use inverted images 
       inverted_img = PIL.ImageOps.invert(img)

       #to convert to YCrCb
       #imgYCC = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB) # or inverted_img 

       # resize the original image to the size mentioned
       img_reshape=inverted_img.resize(size) ## choose: img, imgYCC, inverted_img
       #print(img_reshape.format, img_reshape.size, img_reshape.mode)
       # Location of the resized image as per the label
       augmented_loc= augmented_loc_path + label + "/" + file_name
       aug_dir=augmented_loc_path + label
        
       x = img_to_array(img_reshape)  # this is a Numpy array with shape (3, 32, 32) if size = 32,32
       x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 32, 32) if size = 32,32
        
       # checks if the resized is does not exist already 
       img_reshape.save(augmented_loc)
       # the .flow() command below generates batches of randomly transformed images
       # and saves the results to the directory
       i = 0
       for batch in datagen.flow(x, batch_size=1,
                                 save_to_dir=aug_dir, save_prefix='IMG_AUG', save_format='jpg'):
           i += 1
           if i > 20:
               break  # otherwise the generator would loop indefinitely
        
        
       # converting the image object to 
       np_im = np.array(img_reshape)
       class_number=classes_dict[label] 
       X.append(np_im)
       Y.append(class_number)
       

  except Exception as e:
           print(str(e))
    
X = np.array(X)
Y = np.array(Y)
print(Y.shape)